In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download("tagsets")
from nltk.stem import PorterStemmer
import random
stemmer = PorterStemmer()
#stemmer.stem()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tony\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Tony\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [6]:
def enhance_df(df, lowercase=True, tag_repeat=True, token_adj=True, verbose=False):
    if lowercase:
        df['Token_lower'] = df['Token'].str.lower()
        df['Title_lower'] = df['Title'].str.lower()
    
    df['Tag_adj']=df['Tag']
    df['POS']=None
    
    
    pos_col = df.columns.get_loc("POS")
    tkn_col = df.columns.get_loc("Token")
    tkn_l_col = df.columns.get_loc("Token_lower")
    tag_col = df.columns.get_loc("Tag")
    tag_adj_col = df.columns.get_loc("Tag_adj")
    
    rec = 0
    for i in range(0, len(df)):
        curr=df.iat[i,0]
        if df.iat[i, tag_adj_col]=="":
            last = df.iat[i-1,tag_adj_col]
            if tag_repeat:
                df.iat[i,tag_adj_col]=last
            else:
                if re.match(r".*_(\d)", last):
                    df.iat[i,tag_adj_col]=str(last.split("_")[0] + "_" + str(int(last.split("_")[1])+1))
                else:
                    df.iat[i,tag_adj_col]=str(last+"_2")    
        if curr!=rec:
            rec==curr
            sen = df.iat[i, 1]
            sen = sen.split()
            pos = dict(pos_tag(sen))

        df.iat[i, pos_col] = pos[df.iat[i, tkn_col]]
        
        if verbose:
            if i%1000==0:
                print(i)
    

In [7]:
def add_pos(df, idx=0):
    
    df['POS']=""
    
    pos_col = df.columns.get_loc("POS")
    sen_col = df.columns.get_loc("Title")
    tkn_col = df.columns.get_loc("Token")
    
    rec = 0
    for i in range(0, len(df)):
        curr=df.iat[i,idx]
        
        if curr!=rec:
            rec==curr
            sen = df.iat[i, sen_col]
            sen = sen.split()
            pos = dict(pos_tag(sen))

        df.iat[i, pos_col] = pos[df.iat[i, tkn_col]]

In [59]:
def combine_tokens(df):
    df['Token_adj']=df['Token']
    df['adj_qual']=0
    
    tkn_col = df.columns.get_loc("Token")
    tknadj_col = df.columns.get_loc("Token_adj")
    qual_col = df.columns.get_loc("adj_qual")
    tag_col = df.columns.get_loc("Tag")
    
    for i in range(0, len(df)-1):
        if df.iat[i,qual_col]==0:
            
            if (df.iat[i+1,tag_col]=="") or (df.iat[i+1,tag_col]==None) or (str(df.iat[i+1,tag_col])=='nan'):
                j=1
                bstr = str(df.iat[i,tkn_col])
                while True:
                    if (df.iat[i+j,tag_col]=="") or (df.iat[i+j,tag_col]==None) or (str(df.iat[i+j,tag_col])=='nan'):
                        bstr = bstr + " " + str(df.iat[i+j,tkn_col])
                        j+=1
                    else:
                        break
                for k in range(0, j):
                    df.iat[i+k, tknadj_col] = bstr
                    df.iat[i+k,qual_col]=1
            else:
                #df.iat[i,tknadj_col]=df.iat[i,tkn_col]
                df.iat[i,qual_col]=1
    df = df.drop(columns='adj_qual')
    #return df

In [9]:
def BIOE_Tag(df):
    tag_col = df.columns.get_loc("Tag")
    df['BIO_tag']=''
    bio_col = df.columns.get_loc("BIO_tag")
    
    strt_tag = "B-"
    mid_tag = "I-"
    end_tag = "E-"
    
    for x in range(0, len(df)):
        
        if df.iat[x,tag_col]!="":
            cur_tag = str(df.iat[x,tag_col])
        
        if x<len(df)-1:#can look forward
            if df.iat[x+1,tag_col]!='':#next not blank
                if df.iat[x,tag_col]=="":#current is blank and next not blank, must be end
                    df.iat[x,bio_col] = end_tag+cur_tag
                else:#current not blank, next not blank, stand alone tag
                    df.iat[x,bio_col] = cur_tag
            else: #next is blank
                if df.iat[x,tag_col]=="":#current is blank and next is blank, must be inside
                    df.iat[x,bio_col] = mid_tag+cur_tag
                else:#current not blank, next blank, starting multi-tag
                    df.iat[x,bio_col] = strt_tag+cur_tag
        else:#at the end
            if df.iat[x,tag_col]=="":
                df.iat[x,bio_col] = end_tag+cur_tag
            else:
                df.iat[x,bio_col] = cur_tag

In [10]:
def token_bigrams(df):
    tkn_col = df.columns.get_loc("Token")
    df['Bigrams']=""
    bgram_col = df.columns.get_loc("Bigrams")
    for i in range(0, len(df)-1):
        df.iat[i,bgram_col] = str(df.iat[i,tkn_col]) + " " + str(df.iat[i+1,tkn_col])

In [11]:
def token_trigrams(df):
    tkn_col = df.columns.get_loc("Token")
    df['Trigrams']=""
    tgram_col = df.columns.get_loc("Trigrams")
    for i in range(0, len(df)-2):
        df.iat[i,tgram_col] = str(df.iat[i,tkn_col]) + " " + str(df.iat[i+1,tkn_col]) + " " + str(df.iat[i+2,tkn_col]) 

In [12]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w,p,t,ta,org) for w,p,t,ta,org in zip(s["Token"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag_adj"].values.tolist(),
                                                           s["Bigrams"].values.tolist(),
                                                           s["Token_Org"].values.tolist())]
        
        self.grouped = self.data.groupby("Record Number").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [10]:
# with open("Train_Tagged_Titles.tsv", 'r', encoding='utf-8') as f:
#     filedata = f.readlines()
# f.close()

# df = pd.DataFrame(filedata)
# df = pd.DataFrame(df.iloc[:,0].str.split("\t",expand=True))
# df[3] = df[3].str.replace("\n","")
# df.columns = df.iloc[0]
# df = df.drop(df.index[0])

In [60]:
df = pd.read_csv("data/df_BIOtagged.csv")

In [61]:
combine_tokens(df)
df = df.drop(columns='adj_qual')

In [63]:
df = df.dropna()

In [67]:
df.to_csv("df_tokenadj.csv",index=False)

In [66]:
df[df['Token']!=df['Token_adj']]

,Record Number,Title,Token,Tag,BIO_tag,POS,Token_adj
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand,B-Brand,NNP,LOUIS VUITTON
9,2,LOUIS VUITTON Petit Noe Drawstring Shoulder Ba...,LOUIS,Brand,B-Brand,NNP,LOUIS VUITTON
11,2,LOUIS VUITTON Petit Noe Drawstring Shoulder Ba...,Petit,Model,B-Model,NNP,Petit Noe
20,3,LOUIS VUITTON Damier Azur Pochette Bosphore Sh...,LOUIS,Brand,B-Brand,NNP,LOUIS VUITTON
45,5,Rank AB Vintage Gucci Sherry line PVC Leather ...,Sherry,Product Line,B-Product Line,NNP,Sherry line
...,...,...,...,...,...,...,...
55048,4994,LOUIS VUITTON Retiro Size PM Monogram Beige M4...,LOUIS,Brand,B-Brand,NNP,LOUIS VUITTON
55057,4995,NWT Michael Kors James Zip Clutch Pouch Black ...,Michael,Brand,B-Brand,NNP,Michael Kors
55078,4997,Auth BURBERRY House Check Derby Leather Baby B...,House,Pattern,B-Pattern,NNP,House Check
55082,4997,Auth BURBERRY House Check Derby Leather Baby B...,Baby,Style,B-Style,NNP,Baby Banner


In [3]:
df[(df['Tag']=="Occasion")|(df['Tag']==None)][50:100]

,Record Number,Title,Token,Tag,BIO_tag,POS
6094,554,Large Canvas Shopping Travel Beach Tote Bag an...,Travel,Occasion,B-Occasion,NNP
6356,578,LOUIS VUITTON Keepall 55 M41424 Boston Bag Tra...,Travel,Occasion,Occasion,NNP
6433,585,Kipling Nylon Bori Duffle Bag Travel Dual Carr...,Travel,Occasion,Occasion,NNP
6533,594,Elephant Tote Bag Zendoodle Adult Coloring Tot...,Market,Occasion,Occasion,NNP
6740,612,NEW Gucci Large Cream Bamboo Daily Handbag Cel...,Daily,Occasion,Occasion,NNP
6802,618,Wedding Womens Bags Evening Handbags Vintage C...,Wedding,Occasion,Occasion,VBG
6805,618,Wedding Womens Bags Evening Handbags Vintage C...,Evening,Occasion,Occasion,NNP
6822,619,Luv BETSEY by Betsey JOHNSON Weekender floral ...,Travel,Occasion,Occasion,NNP
6849,621,NWT Vera Bradley Disney Park Where 's Mickey g...,TRAVEL,Occasion,Occasion,NNP
6986,634,Twos Company Designer Womens Twofer Set 2 Trav...,Travel,Occasion,Occasion,NNP


In [136]:
df = df.drop(columns="Tag")
df

,Record Number,Title,Token,BIO_tag,POS
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,B-Brand,NNP
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,E-Brand,NNP
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN,NNP
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type,NNP
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model,NNP
...,...,...,...,...,...
55116,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,Top,No Tag,NNP
55117,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,Closure,No Tag,NNP
55118,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,Retail,No Tag,NNP
55119,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,$,No Tag,$


In [16]:
dfe = pd.read_csv("df_modified1.csv")
dfe

,Record Number,Title,Token,BIO_tag,POS
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,B-Brand,NNP
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,E-Brand,NNP
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN,NNP
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type,NNP
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model,NNP
...,...,...,...,...,...
78370,r_1999,Convertible Rivet LEATHER Knit Bucket Flap Bag...,Casual,Occasion,NNP
78371,r_1999,Convertible Rivet LEATHER Knit Bucket Flap Bag...,BAG,Type,NNP
78372,r_1999,Convertible Rivet LEATHER Knit Bucket Flap Bag...,from,No Tag,IN
78373,r_1999,Convertible Rivet LEATHER Knit Bucket Flap Bag...,S222,No Tag,NNP


In [17]:
def rand_train_sens(df, num, verbose=False):
    sen_col = df.columns.get_loc("Title")
    tkn_col = df.columns.get_loc("Token")
    bio_tag = df.columns.get_loc("BIO_tag")
    pos_col = df.columns.get_loc("POS")
    rand_sen = {"Record Number":[], "Title":[], "Token":[], "BIO_tag":[],"POS":[]}
    
    for x in range(0,num):
        idx = random.randint(0,len(df))

        record = df.iat[idx, 0]
        subset = df[df['Record Number']==record]
        rnum = "r_"+str(x)
        #new_sen = random_sen(subset,rnum)
        
        #rand_sen = {"Record Number":[], "Title":[], "Token":[], "BIO_tag":[],"POS":[]}
        
        title = ""
        for y in range(0,len(subset)):
            tag = subset.iat[y,bio_tag]
            pos = subset.iat[y,pos_col]
            idx = random.choice(df[(df['BIO_tag']==tag)&(df['POS']==pos)].index)
            tkn = list(df['Token'].iloc[[idx]])[0]

            if y<len(subset)-1:
                title+=tkn+" "
            else:
                title+=tkn

            rand_sen['Token'].append(tkn)
            rand_sen['BIO_tag'].append(tag)
            rand_sen['POS'].append(pos)

        for y in range(0,len(subset)):
            rand_sen['Record Number'].append(rnum)
            rand_sen['Title'].append(title)
        
        if verbose:
            if x%100==0:
                print(x)
        
    dff = pd.DataFrame(rand_sen)
    return dff
    

In [18]:
dff = rand_train_sens(df,4000,verbose=True)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


In [19]:
dfe = pd.concat([dfe,dff])

In [20]:
dfe.to_csv("df_modified2.csv",index=False)

In [21]:
dfe

,Record Number,Title,Token,BIO_tag,POS
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,B-Brand,NNP
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,E-Brand,NNP
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN,NNP
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type,NNP
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model,NNP
...,...,...,...,...,...
46665,r_3999,LOUIS PORTLAND Chevron Bebe Grey VIONETTE Bag ...,Bags,Type,NNP
46666,r_3999,LOUIS PORTLAND Chevron Bebe Grey VIONETTE Bag ...,Tote,Type,NNP
46667,r_3999,LOUIS PORTLAND Chevron Bebe Grey VIONETTE Bag ...,#,No Tag,#
46668,r_3999,LOUIS PORTLAND Chevron Bebe Grey VIONETTE Bag ...,Free,No Tag,CD


# Saving embedding dicts

## tags

In [180]:
biotags = list(np.unique(df['BIO_tag']))

In [183]:
biotags

['Accents',
 'B-Accents',
 'B-Brand',
 'B-Character',
 'B-Character Family',
 'B-Closure',
 'B-Color',
 'B-Country/Region of Manufacture',
 'B-Department',
 'B-Fabric Type',
 'B-Features',
 'B-Handle Drop',
 'B-Handle Style',
 'B-Handle/Strap Material',
 'B-Hardware Material',
 'B-MPN',
 'B-Material',
 'B-Measurement, dimension',
 'B-Model',
 'B-No Tag',
 'B-Occasion',
 'B-Pattern',
 'B-Pocket Type',
 'B-Product Line',
 'B-Size',
 'B-Strap Drop',
 'B-Style',
 'B-Theme',
 'B-Trim Material',
 'B-Type',
 'Brand',
 'Character',
 'Character Family',
 'Closure',
 'Color',
 'Country/Region of Manufacture',
 'Department',
 'E-Accents',
 'E-Brand',
 'E-Character',
 'E-Character Family',
 'E-Closure',
 'E-Color',
 'E-Country/Region of Manufacture',
 'E-Department',
 'E-Fabric Type',
 'E-Features',
 'E-Handle Drop',
 'E-Handle Style',
 'E-Handle/Strap Material',
 'E-Hardware Material',
 'E-MPN',
 'E-Material',
 'E-Measurement, dimension',
 'E-Model',
 'E-No Tag',
 'E-Occasion',
 'E-Pattern',
 'E-

In [181]:
tag_emb={}
tag_emb["PAD"]=0
tag_emb["UNK"]=1
idx=2
for t in biotags:
    if not t in tag_emb.keys():
        tag_emb[t]=idx
        idx+=1

In [182]:
with open("bio_tags.txt", 'w', encoding='utf-8') as f:
    for k, v in tag_emb.items():
        f.write(str(k)+"<|>"+str(v)+"\n")
f.close()

## pos

In [173]:
from nltk.data import load
tagdict = load("help/tagsets/upenn_tagset.pickle")
nltkpos = list(tagdict.keys())
dfpos = list(np.unique(df['POS']))
dfpos.extend(nltkpos)

In [175]:
pos_emb = {}
pos_emb["PAD"]=0
pos_emb["UNK"]=1
idx=2
for p in dfpos:
    if not p in pos_emb.keys():
        pos_emb[p]=idx
        idx+=1

In [177]:
with open("pos_tags.txt", 'w', encoding='utf-8') as f:
    for k, v in pos_emb.items():
        f.write(str(k)+"<|>"+str(v)+"\n")
f.close()